### All necessary imports

In [1]:
import urllib.request
from urllib.request import urlopen
import os
import string
import glob
import shutil
import csv
import pprint
import pandas as pd
import numpy as np
from zipfile import ZipFile
from urllib.request import urlopen
from tempfile import NamedTemporaryFile
from shutil import unpack_archive
from io import BytesIO

In [2]:
directory = os.path.dirname(os.getcwd()) 
path =  directory + '/Part_2'
path
#directory

'/Users/sonalichaudhari/Desktop/FALL2017/ADS/Assignment_1/Part_2'

### Downloading the files in zip

In [3]:
# Unzip and downloading the files in the folder

def openUnzip(url):
    with urlopen(url) as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            zfile.extractall(path+'/Log')

In [4]:
file_obj = open(path+"/year.txt","r" )
year ='2014' #file_obj.readline().rstrip()
zipurl = 'http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/'
generalUrl = "http://www.sec.gov/dera/data/PublicEDGAR-log-file-data/"
months = ['01',"02","03","04","05","06","07","08","09","10","11","12"]
for m in range(len(months)):
    if(str(months[m]) is "01" or str(months[m]) is "02" or str(months[m]) is "03"):
        urlAppended = zipurl + str(year) +"/Qtr1/"+"log"+str(year)+str(months[m])+"01.zip"
        #print(urlAppended)
        #openUnzip(urlAppended)
    if(str(months[m]) is "04" or str(months[m]) is "05" or str(months[m]) is "06"):
        urlAppended = zipurl + str(year) +"/Qtr2/"+"log"+str(year)+str(months[m])+"01.zip"
        #print(urlAppended)
        #openUnzip(urlAppended)
    if(str(months[m]) is "07" or str(months[m]) is "08" or str(months[m]) is "09"):
        urlAppended = zipurl + str(year) +"/Qtr3/"+"log"+str(year)+str(months[m])+"01.zip"
        #print(urlAppended)
        #openUnzip(urlAppended)
    if(str(months[m]) is "10" or str(months[m]) is "11" or str(months[m]) is "12"):
        urlAppended = zipurl + str(year) +"/Qtr4/"+"log"+str(year)+str(months[m])+"01.zip"
        #print(urlAppended)
        #openUnzip(urlAppended)

### Reading the CSV in a dataframe

In [5]:
# for name in glob.glob(path+'/Log/*.csv'):
#     with open(name) as csvfile:
#         df = pd.read_csv(name)
#         print(name)

In [235]:
for name in glob.glob('/Users/sonalichaudhari/Desktop/FALL2017/ADS/Assignment_1/Part_2/Log/log20140101.csv'):
    with open(name) as csvfile:
        df = pd.read_csv(name)
        sf = pd.read_csv(name)
        #print(name)

In [240]:
df
sf = df

In [ ]:
# import httpagentparser
# s = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/604.1.38 (KHTML, like Gecko) \
# Version/11.0 Safari/604.1.38"
# browser = httpagentparser.detect(s)

##  Data Wrangling

### Extensions

In [241]:
sf = sf.head(5)
sf
a = sf
a

,ip,date,time,zone,cik,accession,extention,code,size,idx,norefer,noagent,find,crawler,browser
0,108.223.230.dda,2014-01-01,00:00:00,0.0,740670.0,0000905608-00-000038,.txt,200.0,4756.0,0.0,0.0,1.0,10.0,0.0,NaN
1,151.151.7.agj,2014-01-01,00:00:00,0.0,91767.0,0001225208-13-025279,.txt,200.0,5449.0,0.0,0.0,1.0,10.0,0.0,NaN
2,151.151.7.agj,2014-01-01,00:00:00,0.0,91767.0,0001225208-13-025281,.txt,200.0,5466.0,0.0,0.0,1.0,10.0,0.0,NaN
3,151.151.7.agj,2014-01-01,00:00:00,0.0,91767.0,0001225208-13-025278,.txt,200.0,8012.0,0.0,0.0,1.0,10.0,0.0,NaN
4,151.151.7.agj,2014-01-01,00:00:00,0.0,91767.0,0001225208-13-025280,.txt,200.0,5568.0,0.0,0.0,1.0,10.0,0.0,NaN


#### CODE

In [242]:
a['extention'] = np.where(str(a['extention']).find('.') ==0,a['extention'],a['accession']+a['extention'])

/Users/sonalichaudhari/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [185]:
#a

### Zone

In [243]:
m_zone = a['ip'].value_counts().idxmax()
a['zone'] = df['zone'].fillna(m_zone)

/Users/sonalichaudhari/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [186]:
# e = pd.DataFrame(a.groupby('ip').size().rename('cnt'))#.idxmax()[0]
# e
# b= a.iloc[df.groupby(['ip']).apply(lambda x: x['count'].idxmax())]
# b

In [187]:
# z= a.groupby(['ip'], sort=False).idxmax()
# print(z)

In [188]:
# max = a.iloc[a.groupby(['ip']).apply(lambda x: x['cik'].idxmax())]
# a.groupby('ip').count()

### IDX

In [244]:
m_idx = a['idx'].value_counts().idxmax()

In [245]:
a['idx'] = df['idx'].fillna(m_idx)

/Users/sonalichaudhari/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


### Norefer

In [246]:
m_nr = a['norefer'].value_counts().idxmax()
a['norefer'] = df['norefer'].fillna(m_nr)

/Users/sonalichaudhari/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


### Noagent

In [247]:
m_agent = a['noagent'].value_counts().idxmax()
a['noagent'] = df['noagent'].fillna(m_agent)

/Users/sonalichaudhari/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


### Crawler

In [248]:
m_crawler = a['crawler'].value_counts().idxmax()
m_crawler
a['crawler'] = df['crawler'].fillna(m_crawler)

/Users/sonalichaudhari/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


## SUMMARIES

In [249]:
a['size_mean'] = a['size'].mean(axis=0)

/Users/sonalichaudhari/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [250]:
a

,ip,date,time,zone,cik,accession,extention,code,size,idx,norefer,noagent,find,crawler,browser,size_mean
0,108.223.230.dda,2014-01-01,00:00:00,0.0,740670.0,0000905608-00-000038,0000905608-00-000038.txt,200.0,4756.0,0.0,0.0,1.0,10.0,0.0,NaN,5850.2
1,151.151.7.agj,2014-01-01,00:00:00,0.0,91767.0,0001225208-13-025279,0001225208-13-025279.txt,200.0,5449.0,0.0,0.0,1.0,10.0,0.0,NaN,5850.2
2,151.151.7.agj,2014-01-01,00:00:00,0.0,91767.0,0001225208-13-025281,0001225208-13-025281.txt,200.0,5466.0,0.0,0.0,1.0,10.0,0.0,NaN,5850.2
3,151.151.7.agj,2014-01-01,00:00:00,0.0,91767.0,0001225208-13-025278,0001225208-13-025278.txt,200.0,8012.0,0.0,0.0,1.0,10.0,0.0,NaN,5850.2
4,151.151.7.agj,2014-01-01,00:00:00,0.0,91767.0,0001225208-13-025280,0001225208-13-025280.txt,200.0,5568.0,0.0,0.0,1.0,10.0,0.0,NaN,5850.2


In [251]:
a['ip_unique'] = a['ip'].nunique()

/Users/sonalichaudhari/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [262]:
a['abc'] = a['cik'].value_counts()
a

/Users/sonalichaudhari/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,ip,date,time,zone,cik,accession,extention,code,size,idx,norefer,noagent,find,crawler,browser,size_mean,ip_unique,abc
0,108.223.230.dda,2014-01-01,00:00:00,0.0,740670.0,0000905608-00-000038,0000905608-00-000038.txt,200.0,4756.0,0.0,0.0,1.0,10.0,0.0,NaN,5850.2,2,NaN
1,151.151.7.agj,2014-01-01,00:00:00,0.0,91767.0,0001225208-13-025279,0001225208-13-025279.txt,200.0,5449.0,0.0,0.0,1.0,10.0,0.0,NaN,5850.2,2,NaN
2,151.151.7.agj,2014-01-01,00:00:00,0.0,91767.0,0001225208-13-025281,0001225208-13-025281.txt,200.0,5466.0,0.0,0.0,1.0,10.0,0.0,NaN,5850.2,2,NaN
3,151.151.7.agj,2014-01-01,00:00:00,0.0,91767.0,0001225208-13-025278,0001225208-13-025278.txt,200.0,8012.0,0.0,0.0,1.0,10.0,0.0,NaN,5850.2,2,NaN
4,151.151.7.agj,2014-01-01,00:00:00,0.0,91767.0,0001225208-13-025280,0001225208-13-025280.txt,200.0,5568.0,0.0,0.0,1.0,10.0,0.0,NaN,5850.2,2,NaN


In [234]:
a.describe()

,zone,cik,code,size,idx,norefer,noagent,find,crawler,browser,ip_unique
count,5.0,5.000000,5.0,5.000000,5.0,5.0,5.0,5.0,5.0,0.0,0.0
mean,0.0,221547.600000,200.0,5850.200000,0.0,0.0,1.0,10.0,0.0,NaN,NaN
std,0.0,290198.243761,0.0,1250.885766,0.0,0.0,0.0,0.0,0.0,NaN,NaN
min,0.0,91767.000000,200.0,4756.000000,0.0,0.0,1.0,10.0,0.0,NaN,NaN
25%,0.0,91767.000000,200.0,5449.000000,0.0,0.0,1.0,10.0,0.0,NaN,NaN
50%,0.0,91767.000000,200.0,5466.000000,0.0,0.0,1.0,10.0,0.0,NaN,NaN
75%,0.0,91767.000000,200.0,5568.000000,0.0,0.0,1.0,10.0,0.0,NaN,NaN
max,0.0,740670.000000,200.0,8012.000000,0.0,0.0,1.0,10.0,0.0,NaN,NaN
